In [2]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import yaml
from datetime import datetime, date, timedelta

In [2]:
POLYTERASSE_CODE = 45594
HONGG_CODE = 45598
IRCHEL_CODE = 45577

desired_time = "07:20"

FITNESS_CODE = 122920



In [3]:
options = webdriver.ChromeOptions()

# Ignore TLS certificate errors to prevent errors. No special requirement for security here
options.add_argument('--ignore-certificate-errors')

# Sets up the browser to be in incognito mode
options.add_argument('--incognito')

# MOST IMPORTANT PARAMATER : enables the headless mode, namely no window is displayed and everything is ran in the background
# options.add_argument('--headless')

# Forces the driver to be in English for compatability matters
options.add_experimental_option(
    'prefs', {'intl.accept_languages': 'en,en_US'})

driver = webdriver.Chrome(options=options)


In [9]:
BASE_URL = "https://schalter.asvz.ch/"
driver.get(BASE_URL)

In [10]:
sleep(1)
switch_login = "btn-warning"
driver.find_element_by_class_name(switch_login).click();
sleep(1)


In [11]:
textbox = "idd_textbox"

driver.find_element_by_class_name(textbox).click()

In [12]:
sleep(0.2)
x_path = "//div[@savedvalue=\"https://aai-logon.ethz.ch/idp/shibboleth\"]"
driver.find_element_by_xpath(x_path).click()

In [13]:
creds_file = open("creds.yaml")
creds = yaml.load(creds_file, Loader=yaml.FullLoader)

username = creds["username"]
password = creds["password"]

In [14]:
x_path = "//input[@name=\"j_username\"]"
username_field = driver.find_element_by_xpath(x_path)
username_field.send_keys(username)

x_path = "//input[@name=\"j_password\"]"
password_field = driver.find_element_by_xpath(x_path)
password_field.send_keys(password)


In [15]:
elem = driver.find_element_by_xpath("//button[@type=\"submit\"]")
elem.click()
sleep(1)

sleep(4)

In [16]:
tomorrow = date.today() + timedelta(days=3)
d = tomorrow.strftime("%Y-%m-%d")
date_in_url = d + "%20" + desired_time

date_url="2021-04-29%2008:55"


In [17]:
lektionen_url = f"https://asvz.ch/426-sportfahrplan?f[0]=sport:{FITNESS_CODE}&f[1]=facility:{POLYTERASSE_CODE}&date={date_in_url}"

driver.get(lektionen_url)


In [19]:
lecture_x_path = "//*[@id=\"block-asvz-next-content\"]/div/div[1]/div[2]/div[4]/div[1]/div/ul/li[1]/a"
freie_platze_x_path = "//*[@id=\"block-asvz-next-content\"]/div/div[1]/div[2]/div[4]/div[1]/div/ul/li[1]/a/div[3]/div/div/div/div[2]"

freie_platze = True
try:
    driver.find_element_by_xpath(freie_platze_x_path).text[:5].lower() == "freie"
except:
    freie_platze = False

if not freie_platze:
    raise Exception("You cannot register for now (either too early, too late or no more places.")
    exit()
else:
    lecture_url = driver.find_element_by_xpath(lecture_x_path).get_attribute("href")


You cannot register for now (either too early, too late or no more places.


In [55]:
driver.get(lecture_url)
sleep(4)

In [61]:
einschreiben_button = driver.find_element_by_xpath("//*[@id=\"btnRegister\"]")
try:    
    if "einschreiben" in einschreiben_button.text:
        einschreiben_button.click()
        print("You were successfully registered!")
    else:
        print("You are already registered")
except:
    print("Oops... Couldn't find registration button")

You are already registered
